# PyCUDA primal-dual algorithms for TV/TGV-constrained imaging problems — Examples
This notebook's purpose is to reproduce the numerical experiments in the article:
> Kristian Bredies. Recovering piecewise smooth multichannel images by minimization of convex functionals with total generalized variation penalty. _Lecture Notes in Computer Science_, 8293:44-77, 2014. doi:[10.1007/978-3-642-54774-4_3](https://doi.org/10.1007/978-3-642-54774-4_3)

In particular, it demonstrates the implemented primal-dual algorithms for solving variational problems with total variation (TV) as well as total generalized variation (TGV) regularization, i.e., problems of the type
$$ \min_u \ D(u) + R(u) $$
where $D$ is a discrepancy functional associated to an imaging problem and $R$ is either the (discrete) total variation
$$
R(u) = \alpha_1 \mathrm{TV}(u) = \alpha_1 \| \nabla u \|_{1}
$$
for $\alpha_0 > 0$ or the (discrete) total generalized variation of second order
$$
R(u) = \mathrm{TGV}_\alpha^2(u) = \min_w \ \alpha_1 \| \nabla u - w \|_1 + \alpha_0 \| \mathcal{E} w \|_1
$$
for $\alpha_0 > 0$, $\alpha_1 > 0$ and $\mathcal{E}w = \frac12(\nabla u + \nabla u^\perp)$ the symmetrized derivative of the vector field $w$. The computations are performed on regular rectangular grids in two dimensions and with multichannel data such as color images. Please confer the above-referenced publication for more details.

---
## Initialization
First, some necessary modules and functions are imported.

In [ ]:
import pycuda.autoinit
from matplotlib.pyplot import subplots, imread, imshow, cm

---
## Denoising examples
The following example code calls the implemented primal-dual algorithms for image denoising with TV/TGV regularization. This method aims at solving the optimization problem
$$
\min_u \ \frac1p \| u - f \|_p^p + R(u) \ ,
$$
for $f$ a noisy image and $p \in \{1, 2\}$. The corresponding functions are `tv_denoise` and `tgv_denoise` from `denoise_pycuda`. They are called as follows:
```
u = tv_denoise(f, alpha1, norm, maxiter, vis)
u = tgv_denoise(f, alpha1, fac, norm, maxiter, vis)
```
Here, `f` is the noisy image, `alpha1` corresponds to the parameter $\alpha_1$, `fac` determines $\alpha_0 = \alpha_1 \cdot \mathrm{fac}$, `norm` corresponds to $p$, `maxiter` is the number of iterations and `vis` triggers a visualization of the current iterate every `vis`th iteration if positive. Both functions return a denoised image `u`.

In [ ]:
from denoise_pycuda import tv_denoise, tgv_denoise
from numpy import random, clip

### Example 1: Face ($L^2$-denoising)
This example demonstrates the case $p=2$. Gaussian noise with variance $\sigma = 0.2$ is added and the denoising methods are called with $\alpha_1$ that give best PSNR values within a reasonable number of iterations.

In [ ]:
random.seed(14031621)
u_clean = imread("test_data/alinas_eye.png")
f = u_clean + random.randn(*u_clean.shape) * 0.2
u_tv = tv_denoise(f, 0.305, 2, 500, -1)
u_tgv = tgv_denoise(f, 0.288, 2.0, 2, 500, -1)

The results can be visualized via `matplotlib.pyplot`.

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = subplots(2, 2, figsize=(13, 10))
ax1.imshow(u_clean); ax1.axis('off'); ax1.set_title('original image')
ax2.imshow(clip(f, 0, 1)); ax2.axis('off'); ax2.set_title('noisy image')
ax3.imshow(clip(u_tv, 0, 1)); ax3.axis('off'); ax3.set_title('TV denoising')
ax4.imshow(clip(u_tgv, 0, 1)); ax4.axis('off'); _ = ax4.set_title('TGV denoising')

### Example 2: Leaf ($L^2$-denoising)
This example reproduces Fig. 2 in *LNCS 8293:44-77, 2014*. Again, $p=2$ and $\alpha_1$ are chosen such that the results are PSNR-optimal.

In [ ]:
# computation
random.seed(14031621)
u_clean = imread("test_data/wet_leaf.png")
f = u_clean + random.randn(*u_clean.shape) * 0.2
u_tv = tv_denoise(f, 0.31, 2, 500, -1)
u_tgv = tgv_denoise(f, 0.298, 2.0, 2, 500, -1)

# visualization
fig, ((ax1, ax2), (ax3, ax4)) = subplots(2, 2, figsize=(13, 10))
ax1.imshow(u_clean); ax1.axis('off'); ax1.set_title('original image')
ax2.imshow(clip(f, 0, 1)); ax2.axis('off'); ax2.set_title('noisy image')
ax3.imshow(clip(u_tv, 0, 1)); ax3.axis('off'); ax3.set_title('TV denoising')
ax4.imshow(clip(u_tgv, 0, 1)); ax4.axis('off'); _ = ax4.set_title('TGV denoising')

### Example 3: Balloons ($L^1$-denoising)
This example demonstrates the case $p=1$ and reproduces Fig. 3 in *LNCS 8293:44-77, 2014*. Impulsive noise is applied (1/3 of the data is replaced by random values) and the denoising methods are called with PSNR-optimal $\alpha_1$.

In [ ]:
# computation
random.seed(14031621)
u_clean = imread("test_data/balloons2.png")
noise = random.rand(*u_clean.shape)
pattern = random.rand(*u_clean.shape) < 1/3
f = u_clean * (1 - pattern) + noise * pattern
u_tv = tv_denoise(f, 0.86, 1, 1000, -1)
u_tgv = tgv_denoise(f, 0.82, 2.0, 1, 1000, -1)

# visualization
fig, ((ax1, ax2), (ax3, ax4)) = subplots(2, 2, figsize=(10, 10))
ax1.imshow(u_clean); ax1.axis('off'); ax1.set_title('original image')
ax2.imshow(clip(f, 0, 1)); ax2.axis('off'); ax2.set_title('noisy image')
ax3.imshow(clip(u_tv, 0, 1)); ax3.axis('off'); ax3.set_title('TV denoising')
ax4.imshow(clip(u_tgv, 0, 1)); ax4.axis('off'); _ = ax4.set_title('TGV denoising')

---
## Deblurring examples
The implemented primal-dual algorithms aim at deblurring a blurry image by solving a regularized deconvolution problem via the Tikhonov functional minimization 
$$
\min_u \ \frac12 \| u \ast k - f \|_2^2 + R(u) \ ,
$$
where $f$ is the blurry, noisy image and $k$ is the blurring kernel. The corresponding functions are `tv_deblur` and `tgv_deblur` from `deblur_pycuda`. They are called as follows:
```
u = tv_deblur(f, k, alpha1, maxiter, vis)
u = tgv_deblur(f, k, alpha1, fac, maxiter, vis)
```
Here, `f` is the blurry, noisy image, `k` represents the blurring kernel, `alpha1` corresponds to the parameter $\alpha_1$, `fac` determines $\alpha_0 = \alpha_1 \cdot \mathrm{fac}$, `maxiter` is the number of iterations and `vis` triggers a visualization of the current iterate every `vis`th iteration if positive. Both functions return a deblurred image `u`.

In [ ]:
from deblur_pycuda import tv_deblur, tgv_deblur
from linop_pycuda import ConvolutionOperator
from pycuda import gpuarray
from numpy import meshgrid, sum, random, clip

### Example 1: Face ($L^2$-deblurring)
This example reproduces Fig. 4 in *LNCS 8293:44-77, 2014*. The original image is convolved with an out-of-focus kernel with a diameter of 15 pixels and Gaussian noise with variance $\sigma = 0.05$ is added. As before, the $\alpha_1$ are chosen such that the results are PSNR-optimal.

In [ ]:
# blurring kernel and operator
x = range(-7, 8)
[X, Y] = meshgrid(x, x)
mask = (X * X + Y * Y <= 7 * 7).astype('float32')
mask = mask / sum(mask[:])
K = ConvolutionOperator(mask)

# data generation
random.seed(14031621)
u_clean = imread("test_data/alinas_eye512.png")
u_gpu = gpuarray.to_gpu(u_clean.astype('float32').copy(order='F'))
Ku = gpuarray.zeros(K.get_dest_shape(u_gpu.shape), 'float32', order='F')
K.apply(u_gpu, Ku)
f = Ku.get() + random.randn(*Ku.shape) * 0.05

# computation
u_tv = tv_deblur(f, mask, 0.0101, 1000, -1)
u_tgv = tgv_deblur(f, mask, 0.0085, 4.0, 1000, -1)

# visualization
fig, ((ax1, ax2), (ax3, ax4)) = subplots(2, 2, figsize=(13, 10))
ax1.imshow(u_clean); ax1.axis('off'); ax1.set_title('original image')
ax2.imshow(clip(f, 0, 1)); ax2.axis('off'); ax2.set_title('blurred, noisy image')
ax3.imshow(clip(u_tv, 0, 1)); ax3.axis('off'); ax3.set_title('TV deblurring')
ax4.imshow(clip(u_tgv, 0, 1)); ax4.axis('off'); _ = ax4.set_title('TGV deblurring')

### Example 2: Peppers ($L^2$-deblurring)
Another deblurring example. Again, an out-of-focus kernel (diameter: 15 pixels) and additive Gaussian noise ($\sigma = 0.05$) are employed. The $\alpha_1$ are chosen yield PSNR-optimal results.

In [ ]:
# blurring kernel and operator
x = range(-7, 8)
[X, Y] = meshgrid(x, x)
mask = (X * X + Y * Y <= 7 * 7).astype('float32')
mask = mask / sum(mask[:])
K = ConvolutionOperator(mask)

# data generation
random.seed(14031621)
u_clean = imread("test_data/peppers.png")
u_gpu = gpuarray.to_gpu(u_clean.astype('float32').copy(order='F'))
Ku = gpuarray.zeros(K.get_dest_shape(u_gpu.shape), 'float32', order='F')
K.apply(u_gpu, Ku)
f = Ku.get() + random.randn(*Ku.shape) * 0.05

# computation
u_tv = tv_deblur(f, mask, 0.0082, 1000, -1)
u_tgv = tgv_deblur(f, mask, 0.0074, 4.0, 1000, -1)

# visualization
fig, ((ax1, ax2), (ax3, ax4)) = subplots(2, 2, figsize=(10, 10))
ax1.imshow(u_clean); ax1.axis('off'); ax1.set_title('original image')
ax2.imshow(clip(f, 0, 1)); ax2.axis('off'); ax2.set_title('blurred, noisy image')
ax3.imshow(clip(u_tv, 0, 1)); ax3.axis('off'); ax3.set_title('TV deblurring')
ax4.imshow(clip(u_tgv, 0, 1)); ax4.axis('off'); _ = ax4.set_title('TGV deblurring')

---
## Zooming examples
The example code includes primal-dual algorithms for solving zooming problems in the following variational form:
$$
\min_u \ R(u) \qquad \text{subject to} \ \qquad (Ku)_{i,j} = g_{i,j} \quad \forall i=\{0, \ldots, N-1\}, \ j= \{0,\ldots, M-1\} \ ,
$$
where $K$ is a downsampling operator mapping high-resolution images to images of size $N \times M$ and $(g_{i,j})_{i,j}$ are the respective coefficients of a low-resolution image $f$ of size $N \times M$. The high-resolution image $u$ could be of any size greater than $N \times M$. The algorithms implement two downsampling strategies: block averaging and DCT (Discrete Cosine Transform).

Zooming with block averaging as downsampling is available via `tv_zoom` and `tgv_zoom` from `zoom_pycuda`:
```
u = tv_zoom(f, power, alpha1, maxiter, vis)
u = tgv_zoom(f, power, alpha1, fac, maxiter, vis)
```
Zooming with DCT as downsampling is available via `tv_zoom_dct` and `tgv_zoom_dct` from `zoom_pycuda`:
```
u = tv_zoom_dct(f, power, alpha1, maxiter, vis)
u = tgv_zoom_dct(f, power, alpha1, fac, maxiter, vis)
```
Here, `f` is the low-resolution image (in case of DCT-zooming restricted to the size $2^n \times 2^m$ for some positive integers $n$ and $m$), the positive integer `power` indicates a zooming factor of $2^{\mathrm{power}}$, `alpha1` corresponds to the parameter $\alpha_1$, `fac` determines $\alpha_0 = \alpha_1 \cdot \mathrm{fac}$, `maxiter` is the number of iterations and `vis` triggers a visualization of the current iterate every `vis`th iteration if positive. Each function returns a high-resolution image `u` with size $2^{\mathrm{power}}$ times the size of `f`.

In [ ]:
from zoom_pycuda import tv_zoom, tgv_zoom, tv_zoom_dct, tgv_zoom_dct
from linop_pycuda import ZoomingOperator, DCTZoomingOperator
from common_pycuda import enlarge_next_power_of_2
from pycuda import gpuarray
from numpy import clip

### Example 1: Violin (Block averaging zooming)
An image of size 64 x 64 is zoomed by the factor 8 using averaging over 8 x 8 blocks as downsampling model. The parameters $\alpha_1$ are chosen to give best PSNR after 4000 iterations.

In [ ]:
# load and downsample example image
f = imread("test_data/violin.png")
f_gpu = gpuarray.to_gpu(f.astype('float32').copy(order='F'))
K = ZoomingOperator(3)
u = gpuarray.zeros(K.get_dest_shape(f.shape), 'float32', order='F')
K.apply(f_gpu, u)
g = u.get()

# computation
u_tv = tv_zoom(g, 3, 0.13, 4000, -1)
u_tgv = tgv_zoom(g, 3, 0.03, 3.0, 4000, -1)

# visualization
fig, ((ax1, ax2), (ax3, ax4)) = subplots(2, 2, figsize=(10, 10))
ax1.imshow(f); ax1.axis('off'); ax1.set_title('original image')
ax2.imshow(clip(g, 0, 1)); ax2.axis('off'); ax2.set_title('downsampled image')
ax3.imshow(clip(u_tv, 0, 1)); ax3.axis('off'); ax3.set_title('TV zooming')
ax4.imshow(clip(u_tgv, 0, 1)); ax4.axis('off'); _ = ax4.set_title('TGV zooming')

### Example 2: Hand (DCT-zooming)
This example reproduces Fig. 5 in *LNCS 8293:44-77, 2014*. Again, an image of size 64 x 64 is zoomed by the factor 8. The downsampling model extracts the DCT coefficients associated with the lowest frequencies. The parameters $\alpha_1$ are chosen to give best PSNR after 2000 iterations.

In [ ]:
# load and downsample example image
f = enlarge_next_power_of_2(imread("test_data/hand.png"))
f_gpu = gpuarray.to_gpu(f.astype('float32').copy(order='F'))
K = DCTZoomingOperator(3, (f.shape[0] >> 3, f.shape[1] >> 3))
u = gpuarray.zeros(K.get_dest_shape(f.shape), 'float32', order='F')
K.apply(f_gpu, u)
g = u.get() / 8

# computation
u_tv = tv_zoom_dct(g, 3, 0.019, 2000, -1)
u_tgv = tgv_zoom_dct(g, 3, 0.028, 3.8, 2000, -1)

# visualization
fig, ((ax1, ax2), (ax3, ax4)) = subplots(2, 2, figsize=(10, 10))
ax1.imshow(f); ax1.axis('off'); ax1.set_title('original image')
ax2.imshow(clip(g, 0, 1)); ax2.axis('off'); ax2.set_title('downsampled image')
ax3.imshow(clip(u_tv, 0, 1)); ax3.axis('off'); ax3.set_title('TV zooming')
ax4.imshow(clip(u_tgv, 0, 1)); ax4.axis('off'); _ = ax4.set_title('TGV zooming')

---
## Dequantization example
Dequantization aims at reconstructing an image from quantized image data. A primal-dual algorithm is available in the exmaple code that solves the following variational dequantization approach:
$$ \min_u \ \frac1p\| u - f \|_2^2 + R(u) \qquad \text{subject to} \qquad f_{\mathrm{lower}} \leq u \leq f_{\mathrm{upper}} \ , $$
with $f_{\mathrm{lower}}$ and $f_{\mathrm{upper}}$ describing the lower and upper bounds of the pointwise quantization intervals, respectively, and $f = \frac12(f_{\mathrm{lower}} + f_{\mathrm{upper}})$. The algorithm can be called via `tv_dequantize`and `tgv_dequantize` in `dequantize_pycuda`:
```
u = tv_dequantize(lower, upper, alpha1, maxiter, vis)
u = tgv_dequantize(lower, upper, alpha1, fac, maxiter, vis)
```
Here, `lower` and `upper` are the quantization bounds $f_{\mathrm{lower}}$ and $f_{\mathrm{upper}}$, respectively, `alpha1` corresponds to the parameter $\alpha_1$, `fac` determines $\alpha_0 = \alpha_1 \cdot \mathrm{fac}$, `maxiter` is the number of iterations and `vis` triggers a visualization of the current iterate every `vis`th iteration if positive. Both functions return a dequantized image `u`.

In [ ]:
from dequantize_pycuda import tv_dequantize, tgv_dequantize
from pycuda import gpuarray
from numpy import clip, floor, ceil

### Example: Oak leaf ($L^2$-dequantization)
This example reproduces Fig. 6 in *LNCS 8293:44-77, 2014*. The original image is quantized with 6 bins per channel. As before, the $\alpha_1$ are chosen such that the results are PSNR-optimal.

In [ ]:
# load and quantize example image
f = imread("test_data/oak_leaf.png")
lower = floor(f * 6) / 6
upper = ceil(f * 6) / 6

# computation
u_tv = tv_dequantize(lower, upper, 0.49, 2000, -1)
u_tgv = tgv_dequantize(lower, upper, 0.45, 2.0, 2000, -1)

# visualization
fig, ((ax1, ax2), (ax3, ax4)) = subplots(2, 2, figsize=(10, 10))
ax1.imshow(f); ax1.axis('off'); ax1.set_title('original image')
ax2.imshow(clip(1/2*(lower + upper), 0, 1)); ax2.axis('off'); ax2.set_title('quantized image')
ax3.imshow(clip(u_tv, 0, 1)); ax3.axis('off'); ax3.set_title('TV dequantization')
ax4.imshow(clip(u_tgv, 0, 1)); ax4.axis('off'); _ = ax4.set_title('TGV dequantization')

---
## Compressive imaging examples
Compressive imaging aims at reconstructing an image from a few linear measurements by exploiting sparsity of the data in an appropriate sense. The example code implements a primal-dual algorithm for realizing the following variational compressive imaging model:
$$ \min_u \ R(u) \qquad \text{subject to} \qquad Ku = f \ , $$
where $K$ maps the image space onto the measurement data $f$. The algorithm for solving such problems are generic Tikhonov functional minimization/linearly constrained minimization solvers: `tv_tikhonov` and `tgv_tikhonov` from `tikhonov_pycuda`:
```
u = tv_tikhonov(K, f, alpha1, maxiter, vis)
u = tgv_tikhonov(K, f, alpha1, fac, maxiter, vis)
```
Here, $K$ is a general forward operator (an instance of `LinearOperator` from `linop_pycuda`), `f` is the measured data, `alpha1` encodes the regularization parameter and whether Tikhonov functional minimization with $L^2$ discrepancy (`alpha1 > 0` with $\alpha_1 = $ `alpha1`) or linearly constrained minimization (`alpha1 < 0` and $\alpha_1 = $ `-alpha1`) is used, `fac` determines $\alpha_0 = \alpha_1 \cdot \mathrm{fac}$, `maxiter` is the number of iterations and `vis` triggers a visualization of the current iterate every `vis`th iteration if positive. Both functions return an approximation `u` of a solution to the Tikhonov minimization/linearly constrained minimization problem.

For compressive imaging, the forward operator $K$ is realized by `AccumulationOperator` from `linop_pycuda`:
```
K = AccumulationOperator(phi, size)
```
The parameter `phi` corresponds to the compressive imaging matrix and `size` specifies the size of the image. 

In [ ]:
from tikhonov_pycuda import tv_tikhonov, tgv_tikhonov
from linop_pycuda import AccumulationOperator
from pycuda import gpuarray
from numpy import maximum
from scipy.io import loadmat

### Example 1: Mug (single-pixel camera data)

In [ ]:
# read data
Phi = loadmat("compressed_sensing/Phi_64.mat").get("Phi")
f = loadmat("compressed_sensing/Mug_64.mat").get("y")

# construct operator
K = AccumulationOperator(Phi, (64, 64))

# helper function
def reconstruct(samples, alpha, maxiter):
    u_tv = tv_tikhonov(K, f[0:samples], alpha, maxiter=maxiter, vis=-1)
    u_tv = maximum(0.0, u_tv[::-1, :])

    u_tgv = tgv_tikhonov(K, f[0:samples], alpha, maxiter=maxiter, vis=-1)
    u_tgv = maximum(0.0, u_tgv[::-1, :])

    return (samples, u_tv, u_tgv)

# reconstruction
u = [reconstruct(768, -0.001, 15000), reconstruct(384, -0.001, 20000),
     reconstruct(256, -0.001, 30000), reconstruct(192, -0.001, 30000)]

# visualization
fig, ax = subplots(2, 4, figsize=(13, 6.5))
for (i, (samples, u_tv, u_tgv)) in enumerate(u):
    ax[0, i].imshow(u_tv, cmap=cm.gray); ax[0, i].axis('off')
    ax[0, i].set_title(f'TV reconstruction\n{samples} samples')
    ax[1, i].imshow(u_tgv, cmap=cm.gray); ax[1, i].axis('off')
    ax[1, i].set_title(f'TGV reconstruction\n{samples} samples')

### Example 2: Violin (synthetic data)

In [ ]:
# read data
Phi = loadmat("compressed_sensing/Phi_128.mat").get("Phi")

# construct operator
K = AccumulationOperator(Phi, (128, 128))

# generate measurement data
u = imread('test_data/violin_gray128.png')
u_gpu = gpuarray.to_gpu(u[::-1, :].astype('float32').copy(order='F'))
f = gpuarray.zeros(K.get_dest_shape(u.shape), 'float32', order='F')
K.apply(u_gpu, f); f = f.get()

# helper function
def reconstruct(samples, alpha, maxiter):
    u_tv = tv_tikhonov(K, f[0:samples], alpha, maxiter=maxiter, vis=-1)
    u_tv = maximum(0.0, u_tv[::-1, :])

    u_tgv = tgv_tikhonov(K, f[0:samples], alpha, maxiter=maxiter, vis=-1)
    u_tgv = maximum(0.0, u_tgv[::-1, :])

    return (samples, u_tv, u_tgv)

# reconstruction
u = [reconstruct(3072, -10, 40000), reconstruct(1536, -10, 40000),
     reconstruct(1024, -10, 40000), reconstruct(768, -2, 40000)]

# visualization
fig, ax = subplots(2, 4, figsize=(13, 6.5))
for (i, (samples, u_tv, u_tgv)) in enumerate(u):
    ax[0, i].imshow(u_tv, cmap=cm.gray); ax[0, i].axis('off')
    ax[0, i].set_title(f'TV reconstruction\n{samples} samples')
    ax[1, i].imshow(u_tgv, cmap=cm.gray); ax[1, i].axis('off')
    ax[1, i].set_title(f'TGV reconstruction\n{samples} samples')